In [1]:
using CairoMakie, ColorSchemes, Optim, LinearAlgebra, QuantEcon

In [26]:
include("crra_code.jl")
include("ces_code.jl")

hpi_CES (generic function with 2 methods)

In [2]:
function invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
    (; na, nz) = pars
    for i in 1:na
        for j in 1:nz
            if policy[i,j] <+ Avals[1]
                M[i,j,1,:] = Π[j,:]
            elseif policy[i,j] >= Avals[end]
                M[i,j,end,:] = Π[j,:]
            else
                index = findfirst(x -> x > policy[i,j], Avals)
                π = (Avals[index] - policy[i,j]) / (Avals[index] - Avals[index-1])
                M[i,j,index-1,:] = π * Π[j,:]
                M[i,j,index,:] = (1-π) * Π[j,:]
            end
        end
    end
    O = reshape(M, n, n)
    for i in 1:na
        O[i,:] = O[i,:] / sum(O[i,:])
    end
    X = reduce(vcat, [I(n) - O', ones(1,n)])
    Y = [zeros(n);1]
    Inv = X \ Y
    Inv[Inv .< 0] .= 0
    Inv = Inv / sum(Inv)
    return Inv
end

function labour_supply(Π, Zvals, pars)
    (; nz) = pars
    L = zeros(nz)
    L2 = reduce(vcat, [I(nz) - Π', ones(1,nz)])
    L3 = [zeros(nz);1]
    L = L2 \ L3
    L[L .< 0] .= 0
    L = L / sum(L)
    L = L' * Zvals
    return L
end

labour_supply (generic function with 1 method)

In [17]:
ces_model = create_model_CES()

(A = 0.01, α = 0.33, δ = 0.08, r_lb = 0.0, r_ub = 0.1, r_iter = 0.0, ϕ = 0.0, w = 0.01, β = 0.9, γ = 2.0, nz = 15, ρ = 0.9, μ = 0.0, σ = 0.003, θ = 3.0, na = 101, lb = 0.0, ub = 1000.0, max_iter = 1000, how_iter = 25, print_skip = 100, toler = 4.0e-7, toler_prices = 0.001, max_iter_prices = 100, print_skip_prices = 1, agrid = [0.0, 0.0010000000000000002, 0.008000000000000002, 0.027, 0.06400000000000002, 0.12500000000000003, 0.216, 0.3430000000000001, 0.5120000000000001, 0.729  …  753.5710000000001, 778.6880000000001, 804.3570000000001, 830.5839999999998, 857.3749999999999, 884.736, 912.673, 941.1919999999999, 970.2989999999999, 1000.0], zgrid = [0.03457959025122844, 0.05594303083042085, 0.09050490985451279, 0.14641928737474275, 0.23687784176339377, 0.3832221350379456, 0.6199786509779679, 1.003004504503377, 1.6226656102869836, 2.625156388616362, 4.246990890177603, 6.870802707018247, 11.115618342377728, 17.982901911474734, 29.092826975250997], Π = [6.103515625e-5 0.0008544921875 … 0.0008

In [24]:
function equilibrium_vfi_CES(pars)
    (; agrid, zgrid, Π, na, nz, toler_prices, max_iter_prices, print_skip_prices, r_lb, r_ub, w, β, A, α, ϕ, δ) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)
    v_init = zeros(na,nz)
    policy = similar(v_init)
    wealth = zeros(na,nz)
    L = labour_supply(Π, zgrid, pars)
    Ks, Kd = 1, 1
    store_Kd = []
    store_Ks = []
    store_r = []
    iter = 0
    error = toler_prices + 1
    if iter == 0
        println("Iterating on prices...")
    end
    while ((error > toler_prices) && (iter < max_iter_prices))
        println("////////////////////")
        println("Price Iteration: $iter")
        r_iter = (pars.r_lb + pars.r_ub) / 2
        push!(store_r, r_iter)
        Kd = ((A^α * L ^ (1-α)) / (r_iter + δ))^1/(1-α)
        push!(store_Kd, Kd)
        w_iter = (1-α) * A * (Kd/L)^α
        Φ = w_iter * ((minimum(zgrid))/r_iter)
        if ϕ > 0
            ϕ_iter = min(Φ, minimum(zgrid))
            pars = (; pars..., ϕ = ϕ_iter, r_iter = r_iter, w = w_iter)
        else
            pars = (; pars..., r_iter = r_iter, w = w_iter)
        end
        v_new, policy = vfi_CES(v_init, policy, Π, zgrid, agrid, pars)
        Invariant = invariant_distribution(M, O, X, Y, Inv, policy, Π, agrid, n, pars)
        G = reshape(policy .- ϕ, n, 1)
        Ks = dot(Invariant', G)
        push!(store_Ks, Ks)
        diff = Ks - Kd
        error = abs(diff)
        if diff > 0
            pars = (; pars..., r_ub = r_iter)
        else
            pars = (; pars..., r_lb = r_iter)
        end
        if iter % print_skip_prices == 0
            println("%%%%%%%%%%%%%%%%%%%%")
            println("Iter = $iter, Ks = $Ks, Kd = $Kd, diff = $diff")
            println("%%%%%%%%%%%%%%%%%%%%")
        end
        v_init .= v_new
        iter += 1
    end
    r_final = (pars.r_lb + pars.r_ub) / 2
    w_final = (1-α) * A * (Kd/L)^α
    Invariant = reshape(Invariant, na, nz)
    for i in 1:na
        for j in 1:nz
            wealth[i,j] = w_final * exp(Zvals[j]) + (1 + r_final) * Avals[i]
        end
    end
    println("%%%%%%%%%%%%%%%%%%%%")
    println("r = $r_final, w = $w_final")
    println("%%%%%%%%%%%%%%%%%%%%")
    return v_init, policy, Invariant, wealth
end

equilibrium_vfi_CES (generic function with 1 method)

In [27]:
v_out, policy_out, Invariant_out, wealth_out = equilibrium_vfi_CES(ces_model)

Iterating on prices...
////////////////////
Price Iteration: 0


ErrorException: type NamedTuple has no field maxiter